## Guided session 2

This is the first notebook for the second session of the [Machine Learning workshop series at Harvey Mudd College](http://www.aashitak.com/ML-Workshops/).

Main topics for today's session:
* Split-apply-combine operations by grouping rows of a dataframe
* Encoding categorical variables
* Concatentating and merging dataframes

In [1]:
import pandas as pd
import re # For regular expressions

In today's guided session, we will continue exploring the [Titanic dataset from Kaggle](https://www.kaggle.com/c/titanic). Let us set *Passengerid* as the index.

In [2]:
path = 'titanic/'
df = pd.read_csv(path + 'train.csv') 
df = df.set_index('PassengerId')
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


[Description for the columns](https://www.kaggle.com/c/titanic/data) is as follows.  

|Variable|	Definition|	Key|   
|:---  |:--- |:---|
|PassengerId| Passenger ID |
|Survived| 	Survival|	0 = No, 1 = Yes |
|Pclass	|Ticket class|	1 = 1st, 2 = 2nd, 3 = 3rd|
|Sex	|Sex|	
|Age	|Age in years	|
|SibSp	|# of siblings / spouses aboard the Titanic	|
|Parch	|# of parents / children aboard the Titanic	|
|Ticket	|Ticket number	|
|Fare	|Passenger fare	|
|Cabin	|Cabin number	|
|Embarked	|Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton|

### 1. [GroupBy object](https://pandas.pydata.org/pandas-docs/version/0.22/groupby.html)
The below code is a repetition from the exercises in the previous session to create a new column named *Title* that is derived from the column *Name* using regular expressions.

In [3]:
df['Title'] = df['Name'].map(lambda name: re.findall("\w+[.]", name)[0])

title_dictionary = {'Ms.': 'Miss.', 'Mlle.': 'Miss.', 
              'Dr.': 'Rare', 'Mme.': 'Mr.', 
              'Major.': 'Rare', 'Lady.': 'Rare', 
              'Sir.': 'Rare', 'Col.': 'Rare', 
              'Capt.': 'Rare', 'Countess.': 'Rare', 
              'Jonkheer.': 'Rare', 'Dona.': 'Rare', 
              'Don.': 'Rare', 'Rev.': 'Rare'}

df['Title'] = df['Title'].replace(title_dictionary)

df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.


As we saw in the last session, the *Age* column has a lot of missing values. It can be filled with the median (or mean) age of all passengers, but we can also make use of the derived *Title* column to fill the age based on the title of the passenger. For this, it would be helpful to group the passengers based on their title to get the median age for each group and then fill the missing age values for passengers with the median age for their title.

We can use [`groupby()`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.groupby.html) to group the rows of the dataframe based on a column(s), say *Title*.

In [4]:
df.groupby('Title')

The resulting object is not a dataframe but there are multiple ways to derive a dataframe from the grouped object.

One of the ways to derive a dataframe from a groupby object is by aggregation, that is computing a summary statistic (or statistics) about each group. For example, we can get the median values for the columns in each group of titles.

In [5]:
df.groupby('Title').median()

,Survived,Pclass,Age,SibSp,Parch,Fare
Title,,,,,,
Master.,1,3,3.5,1,1,29.06250
Miss.,1,3,21.0,0,0,15.74170
Mr.,0,3,30.0,0,0,9.41665
Mrs.,1,2,35.0,1,0,26.00000
Rare,0,1,48.5,0,0,27.72080


In [6]:
df.groupby('Title').mean()

,Survived,Pclass,Age,SibSp,Parch,Fare
Title,,,,,,
Master.,0.575000,2.625000,4.574167,2.300000,1.375000,34.703125
Miss.,0.702703,2.291892,21.845638,0.702703,0.540541,43.800092
Mr.,0.158301,2.407336,32.347118,0.287645,0.152510,24.528159
Mrs.,0.792000,2.000000,35.898148,0.696000,0.832000,45.138533
Rare,0.347826,1.347826,45.545455,0.347826,0.086957,37.169748


The median age vary greatly for each group ranging from 3.5 to 48 years.

The most common way to derive a dataframe from a groupby object is by transformation. We create a new column *MedianAge* which consists of the groupwise median age depending on the passengers' title using [`transform()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html).

In [8]:
df['MedianAge'] = df.groupby('Title')['Age'].transform("median")
df.head(15)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,MedianAge
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.,30.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.,35.0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,21.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.,35.0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.,30.0
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr.,30.0
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr.,30.0
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master.,3.5
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs.,35.0


Now we fill in the missing values in the *Age* column using the values in the *MedianAge* column.

In [11]:
df['Age'] = df['Age'].fillna(df['MedianAge'])
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,MedianAge
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.,30.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.,35.0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,21.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.,35.0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.,30.0


We drop off the *MedianAge* column since we no longer need it.

In [12]:
df = df.drop('MedianAge', axis=1)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.


Let us check for the missing values. There are none in the *Age* column!

In [13]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
Title         0
dtype: int64

### 2. Encoding categorical variables
Let us check the datatype of each column. Hint: Use [`dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html).

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
Title        object
dtype: object

There are two columns with `object` datatype - *Sex* and *Embarked*. These two along with *Pclass* are categorical variables. The feature *Pclass* has an innate order in its categories and hence, is ordinal, whereas *Sex* and *Embarked* are inordinal categorical variables. Most machine learning models require the features or input variables to be numerical. One way to accomplish that is to encode the categories with numbers.

Convert the gender values to numerical values 0 and 1. Hint: Use [`replace`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html) with the suitable dictionary. 

Check the datatypes again and make note of datatype for the column *Sex*. 

In [17]:
df.dtypes

Survived      int64
Pclass        int64
Name         object
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
Title        object
dtype: object

What can go wrong with randomly assigning numbers to categories?

There are two kinds of categorical variables - ordinal and inordinal based on whether the categories possess an inherent order or not.

For example, passengers' ticket class `Pclass` is ordinal whereas gender is inordinal.

Numbers have a natural order and so do ordinal categories. Number also possess certain other characteristics that ordinal categories do not. For example, the difference between the numbers 1 and 2 is the same as the difference between the numbers 2 and 3 but the same cannot be said for ordinal categories. 
$$ 2-1 == 3-2$$
So, converting categories to numbers means adding untrue assumptions that may or may not adversely affect our model. 

For this reason, the prefered method is one-hot encoding. In this method, we build a one-hot encoded vector with dimension equal to the number of classes in the categories. This vector consists of all 0's except for a 1 corresponding to the class of the instance. For example, the *Embarked* column will have one-hot encoded vectors of [1,0,0], [0,1,0] and [0,0,1] for the three ports. This means that we will have three columns for the *Embarked* columns - one for each port and the values for these columns would simply be 1 or 0.

One-hot encoding is accomplished in pandas using [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) as given below. It simply creates a column for each class of a categorical variable.

In [19]:
pd.get_dummies(df['Embarked']).head()

,C,Q,S
PassengerId,,,
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1
5,0,0,1


We want the column names to be `'Port_C', 'Port_Q', 'Port_S'`. Copy the above code with `get_dummies` and modify it to [make use of the `prefix` keyword](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) to alter the column names. Next, save this to a new dataframe named `port_df`.

In [21]:
port_df.head()

,Port_C,Port_Q,Port_S
PassengerId,,,
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1
5,0,0,1


To add this dataframe of two new columns to the original dataframe, we can use `concat` with `axis=1`.

In [23]:
df = pd.concat([df, port_df], axis=1)

Now check that the new columns are added. 

In [24]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Port_C,Port_Q,Port_S
PassengerId,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.,0,0,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.,1,0,0
3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,0,0,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,Mrs.,0,0,1
5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,Mr.,0,0,1


Next, drop the column for *Embarked*. 

Note: if you run the above cell more than once, it will give an error, since the column *Embarked* is no more present in the dataframe for the code to work. 

Next, we check the columns in our dataframe.

In [27]:
df.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Title', 'Port_C', 'Port_Q', 'Port_S'],
      dtype='object')

The expected output is  
```Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Port_C', 'Port_Q', 'Port_S'], dtype='object')```

Notes:
- One of the columns in the one-hot encoding obtained in the above manner is always redundant. In case of features with just two classes such as gender in our dataset, one-hot encoding is not truly useful. One of its column is same as what we obtained by simply replacing classes with 0 and 1 and the other is redundant.  
- The main disadvantage of using one-hot encoding is the increase in the number of features that can negatively affect our model which we will discuss in the later sessions.


### Acknowledgement:
* [Titanic dataset from Kaggle](https://www.kaggle.com/c/titanic) dataset openly available in Kaggle is used in the exercises.

**Note:**
The solutions for this exercise can be found [here](https://github.com/AashitaK/ML-Workshops/blob/master/Session%202/Guided%20session%202.ipynb).

### Next step:

Please proceed to the [hands-on exercises](https://www.kaggle.com/aashita/exercise-2).